In [ ]:
import rootutils

rootutils.setup_root('.', pythonpath=True)

In [ ]:
import matplotlib.pyplot as plt
import cv2
import sys
import torch

from shift_ocr.detectors.fast import FASTDetector
from shift_ocr.recognizer.crnn import CRNNRecognizer
from shift_ocr.postprocessor.simple import SimplePostprocessor

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
vocab = "АӘБВГҒДЕЁЖЗИЙКҚЛМНҢОӨПРСТУҰҮФҺЦЧШЩЪЫІЬЭЮЯаәбвгғдеёжзийкқлмнңоөпрстуұүфһцчшщъыіьэюяABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789!$#()?-.,:;@%&*+=[]{}"

In [ ]:
detector = FASTDetector(
            cuda=False,
            pretrained=True,
            assume_straight_pages=False
        )

In [ ]:
recognizer_no_filter = CRNNRecognizer(
    cuda=False,
    pretrained=False,
    filter_ctc=False,
    vocab=vocab,
    backbone="vgg16",
    weights_path=r"D:\code\recognition-symbols-core\assets\models\model_vgg16_lstm_ctc-True_best.pth"
)

recognizer_filter = CRNNRecognizer(
    cuda=False,
    pretrained=False,
    filter_ctc=True,
    vocab=vocab,
    backbone="vgg16",
    weights_path=r"D:\code\recognition-symbols-core\assets\models\model_vgg16_lstm_ctc-True_best.pth"
)

postprocessor = SimplePostprocessor()

In [ ]:
img_ru = cv2.imread('img_ru.png')
img_eng = cv2.imread('img_eng.png')
img_kz = cv2.imread('img_kz.png')

In [ ]:
recognizer_filter.net

In [ ]:
ru_detections = detector.detect(img_ru)
eng_detections = detector.detect(img_eng)
kz_detections = detector.detect(img_kz)

post_ru_detections = postprocessor.postprocessing(ru_detections, img_ru)
post_eng_detections = postprocessor.postprocessing(eng_detections, img_eng)
post_kz_detections = postprocessor.postprocessing(kz_detections, img_kz)

In [ ]:
from shift_ocr.utils.BBoxVisualizer import BoundingBoxVisualizer

BoundingBoxVisualizer.show_image(img_ru, post_ru_detections)
BoundingBoxVisualizer.show_image(img_eng, post_eng_detections)
BoundingBoxVisualizer.show_image(img_kz, post_kz_detections)

In [ ]:
ru_ocr_detections = recognizer_filter.recognize(post_ru_detections)
eng_ocr_detections = recognizer_filter.recognize(post_eng_detections)
kz_ocr_detections = recognizer_filter.recognize(post_kz_detections)

In [ ]:
from IPython.display import display, HTML

html_output = ""
for pred in ru_ocr_detections:
    # Сохраняем изображение во временный файл
    import tempfile
    from PIL import Image
    
    with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as tmp:
        Image.fromarray(pred.crop).save(tmp.name)
        html_output += f"""
        <div style="margin: 10px; border: 1px solid #ccc; padding: 10px; float: left;">
            <img src="{tmp.name}" style="max-height: 200px;"/>
            <p style="text-align: center; word-wrap: break-word; width: 200px;"><b>{pred.text}</b></p>
        </div>
        """
    
display(HTML(html_output))

# Удаляем временные файлы
for pred in ru_ocr_detections:
    pass  # Файлы будут автоматически удалены при закрытии временного файла